In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
from tensorflow.python.ops import rnn, rnn_cell
from yahoo_finance import Share

In [2]:
day_len = 9
H = 256 # number of hidden layer neurons
D = 9 # input dimensionality
C = 9 # ouput classes 
n_input = 16 # MNIST data input (img shape: 2*2)
n_steps = 16 # timesteps

### get 2330 stock data from yahoo-finance

In [3]:
stock = Share('2330.TW')
today = datetime.date.today()
stock_data = stock.get_historical('2016-01-01', str(today))
stock_data.reverse()
print len(stock_data)

239


### delete the zero-volumn data

In [4]:
i = 0
while( i < len(stock_data)):
    if (int(stock_data[i].get('Volume')) <= 0):
        stock_data.remove(stock_data[i])
        i = -1
    i += 1
print 'data without zero volumn', len(stock_data)

data without zero volumn 221


### training and testing data

In [5]:
data_X = np.zeros((len(stock_data)-day_len,day_len), dtype=np.float)
data_Y = np.ones((len(stock_data)-day_len,1), dtype = np.float)

for i in xrange(0, len(data_X)):
    for j in xrange(0, day_len):
        data_X[i,j] = float(stock_data[i+j].get('Close')) - float(stock_data[i].get('Close'))
    data_Y[i,0] = float(stock_data[i+day_len].get('Close')) - float(stock_data[i].get('Close'))
    
len_train = int(len(data_X) * 0.8)
len_test = len(data_X) - len_train

train_X = data_X[0:len_train]
train_Y = data_Y[0:len_train]
test_X = data_X[len_train::]
test_Y = data_Y[len_train::]

In [6]:
def autoencoder():
    # placeholder
    input_x = tf.placeholder(tf.float32, [None,D], name="input_x")
    
    weights = {
        'encoder_h': tf.Variable(tf.random_normal([D, H])),
        'decoder_h': tf.Variable(tf.random_normal([H, C]))
    }
    biases = {
        'encoder_b': tf.Variable(tf.random_normal([H])),
        'decoder_b': tf.Variable(tf.random_normal([C])),
    }
    
    # Encoder Hidden layer with sigmoid activation
    input_x0 = tf.nn.sigmoid(tf.add(tf.matmul(input_x, weights['encoder_h']),
                                   biases['encoder_b']))
    print input_x0.get_shape()
    input_x1 = tf.reshape(input_x0, [-1,n_steps, n_input])
    input_x2 = tf.transpose(input_x1, [1, 0, 2])
    input_x3 = tf.reshape(input_x2, [-1, n_input])
    input_x4 = tf.split(0, n_steps, input_x3)
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=1.0)
    outputs, states = rnn.rnn(lstm_cell, input_x4, dtype=tf.float32)
    
    score = tf.nn.sigmoid(tf.add(tf.matmul(outputs[-1], weights['decoder_h']),
                                   biases['decoder_b']))
    
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(score-input_x),reduction_indices=[1]))
    
    return{'input_x': input_x, 'score': score, 'loss': loss}

In [7]:
ae = autoencoder()
#train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(ae['loss']) 
train_step = tf.train.AdagradOptimizer(learning_rate=0.001).minimize(ae['loss']) 
init = tf.initialize_all_variables()

(?, 256)


In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    fig1 = plt.figure()
    fig2 = plt.figure()
    ax1 = fig1.add_subplot(1,1,1)
    ax2 = fig2.add_subplot(1,1,1)
    ax1.plot(train_X[:,8], 'b.')
    ax2.plot(test_X[:,8], 'b.')
    plt.ion()
    plt.show()

    for i in range(50000):
    
        # training
        sess.run(train_step, feed_dict={ae['input_x']: train_X})
        
        if i % 50 == 0:
            # to visualize the result and improvement
            try:
                ax1.lines.remove(lines1[0])
            except Exception:
                pass
            
            print i
            print 'loss:', sess.run(ae['loss'], feed_dict={ae['input_x']:train_X})
            
            train_prediction = sess.run(ae['score'], feed_dict={ae['input_x']: train_X})
            #print train_prediction.reshape([len(train_prediction)])
            
            # plot the prediction
            ax1.set_ylim(min(np.min(train_prediction),np.min(train_X))*0.95,max(np.max(train_prediction), np.max(train_X))*1.05)
            lines1 = ax1.plot(train_prediction[:,4], 'r-', lw=1)
                
            
            # testing
            try:
                ax2.lines.remove(lines2[0])
            except Exception:
                pass
            
            test_prediction = sess.run(ae['score'], feed_dict={ae['input_x']: test_X})
            print test_prediction

            # plot the prediction
            ax2.set_xlim(min(np.min(test_prediction),np.min(test_X))*0.95,max(np.max(test_prediction), np.max(test_X))*1.05)
            lines2 = ax2.plot(test_prediction[:,4], 'r-', lw=1)
            plt.pause(0.1)

0
loss: 134.65
[[ 0.61260855  0.97577685  0.95942819  0.13708012  0.96903336  0.79835647
   0.93516338  0.97026169  0.98459375]
 [ 0.91165775  0.97585219  0.90026033  0.0995242   0.97844875  0.90962273
   0.97100508  0.99668092  0.98775417]
 [ 0.7359879   0.99109823  0.87770975  0.03396671  0.94256425  0.91443181
   0.98608691  0.97360206  0.9614206 ]
 [ 0.87042487  0.97299337  0.95190966  0.12778808  0.97729629  0.90976489
   0.94366604  0.99241698  0.97810537]
 [ 0.81347936  0.94092757  0.9237451   0.22106457  0.95884371  0.6864419
   0.91746014  0.99173224  0.96967757]
 [ 0.737221    0.98048955  0.95008689  0.07610331  0.95374662  0.90086907
   0.98481834  0.98339313  0.98700804]
 [ 0.91218805  0.88728201  0.9277966   0.26504061  0.97041571  0.78289711
   0.81067276  0.99521142  0.95630455]
 [ 0.80245501  0.98931247  0.90164691  0.06952655  0.9562977   0.91704601
   0.98754275  0.99327177  0.98452759]
 [ 0.7732448   0.97686225  0.96390313  0.10236914  0.96422666  0.8980177
   0.9611

/usr/lib/pymodules/python2.7/matplotlib/backend_bases.py:2407: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


50
loss: 117.32
[[  2.84292307e-02   9.00294006e-01   9.77613926e-01   9.92973924e-01
    9.98781383e-01   9.86589074e-01   9.89412487e-01   9.85447168e-01
    9.87412453e-01]
 [  1.03794239e-01   6.92443788e-01   8.68619800e-01   9.78241086e-01
    9.92891073e-01   9.93539274e-01   9.94242370e-01   9.98850703e-01
    9.89305735e-01]
 [  4.23160978e-02   9.44057941e-01   9.59391654e-01   9.64599729e-01
    9.93827403e-01   9.94006276e-01   9.97935534e-01   9.90176618e-01
    9.68844235e-01]
 [  8.03235844e-02   7.35073328e-01   7.52824903e-01   9.22131598e-01
    9.82405722e-01   9.86403108e-01   9.73196447e-01   9.93377864e-01
    9.63964701e-01]
 [  3.80112305e-02   2.16758959e-02   5.96204847e-02   3.20542865e-02
    3.37094776e-02   5.87696992e-02   1.45788044e-01   6.52333677e-01
    3.69607598e-01]
 [  4.71746176e-02   9.42409337e-01   9.84496713e-01   9.95316505e-01
    9.98914719e-01   9.97351885e-01   9.99143362e-01   9.97730434e-01
    9.95299816e-01]
 [  7.85252675e-02   1.6

KeyboardInterrupt: 